**BART Trained on XL SUM**

**Packages**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math

from datasets import load_dataset
import evaluate

import inspect

#let's make longer output readable without horizontal scrolling
from pprint import pprint

import warnings

import regex as re

import os, re
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# These auto classes load the right type of tokenizer and model based on a model name
from transformers import AutoTokenizer, TFAutoModel
from transformers import pipeline
from transformers import AutoModel

/home/ubuntu/w266/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Necessary Functions**

In [2]:
rouge = evaluate.load('rouge')

In [3]:
chrf = evaluate.load("chrf")

In [4]:
def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

**BART Resources**

https://github.com/huggingface/transformers/blob/main/examples/pytorch/summarization/run_summarization.py

https://www.databricks.com/blog/2023/03/20/fine-tuning-large-language-models-hugging-face-and-deepspeed.html

https://gitlab.com/nicolalandro/summarization

**Untrained Model**

In [85]:
%pwd
%cd w266_project
%pwd

[Errno 2] No such file or directory: 'w266_project'
/home/ubuntu/w266_project


'/home/ubuntu/w266_project'

In [86]:
df = pd.read_csv('Datasets/xl_sum_sample_val.csv')
df.head(5)

,text,summary
0,Anthony ZurcherNorth America reporter@awzurche...,On day three of public hearings in the impeach...
1,It made a net profit of $281m (£185m) in the t...,"Yum Brands, owner of KFC and Pizza Hut restaur..."
2,Police sources told local media that the boy h...,Four members of the same family have been arre...
3,Zelda Perkins told the Financial Times she sig...,A British former assistant of Harvey Weinstein...
4,Bus workers walked out on Monday over changes ...,Bus drivers in Jersey have agreed to meet with...


In [9]:
dft = pd.read_csv('Datasets/xl_sum_sample_test.csv')
#dft.head(5)

In [13]:
#remember to push new model to huggingface repo after running prev cell so that this cell uses the most recent model!!!!!!!

summarizer = pipeline("summarization", model="facebook/bart-base")

In [15]:
bart_scores(summarizer, dft)

/home/ubuntu/w266/lib/python3.8/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Your max_length is set to 256, but you input_length is only 86. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 256, but you input_length is only 92. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 256, but you input_length is only 193. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Your max_length is set to 256, but you input_length is only 254. You might consider decreasing max_length manually, e.g. summarizer

Last Article By Alice EvansBBC News 1. Value for money For the men's World Cup last summer, your pockets had to be bulging with Russian roubles if you wanted a chance to see football come home (or not) with your own eyes. The best seats in the house for the France v Croatia men's final cost an eye-watering 66,000 roubles (£824.44). But don't despair. If you don't have a spare 800 quid jangling around, it's still possible for you to go to a World Cup final. The most expensive seats in the women's final at the Stade de Lyon on 7 July are more than 10 times cheaper than in the men's tournament at £75.12 (84EUR). At the Women's World Cup in France this year, you can actually watch a game for as little as £8.05 (9EUR). C'est magnifique, non? This pricing imbalance manifests itself in merchandise, too. Perhaps to Nike's credit, the official shirts of the men's and women's World Cups both cost £89.95. However, while it is free to add the name of a Lioness to the women's shirt, it costs £13 to

In [89]:
def bart_untrained(dataframe):

    bart_r1 = []
    bart_r2 = []
    bart_rL = []
    bart_rLs = []
    bart_chrf = []

    for i in range(int(len(dataframe['text']))):

        candidate = summarizer(dataframe['text'][i], 
                               truncation = True, #truncated to first 1024 words, because that is all the model can handle
                               max_length = 256, # same as pegasus
                               min_length = 0, 
                                )[0]
        candidate = [candidate['summary_text']]
        #pprint(candidate[0], compact=True)

        ref = [dataframe['summary'][i]]

        results = rouge.compute(predictions=candidate,
                                references=ref)

        bart_r1.append(results['rouge1'])
        bart_r2.append(results['rouge2'])
        bart_rL.append(results['rougeL'])
        bart_rLs.append(results['rougeLsum'])

        results = chrf.compute(predictions=candidate,
                                references=ref)

        bart_chrf.append(results['score'])

    print('rouge1 average :', np.mean(bart_r1))
    print('rouge2 average :', np.mean(bart_r2))
    print('rougeL average :', np.mean(bart_rL))
    print('rougeLs average :', np.mean(bart_rLs))
    print('chrf average :', np.mean(bart_chrf))
        
    #     if i in np.arange(0, 2000, 100):
    #         data = {'rouge1': bart_r1, 'rouge2': bart_r2, 'rogueL': bart_rL, 'rogueLs': bart_rLs, 'chrf': bart_chrf}
    #         scores = pd.DataFrame(data)
    #         scores.to_csv(r'BART_trained_0_scores.csv', index=False)
    #         print(i)

    # data = {'rouge1': bart_r1, 'rouge2': bart_r2, 'rogueL': bart_rL, 'rogueLs': bart_rLs,'chrf': bart_chrf}
    # scores = pd.DataFrame(data)
    # scores.to_csv(r'BART_trained_0_scores.csv', index=False)

All Categories

In [90]:
bart_untrained(dft)

/home/ubuntu/w266/lib/python3.8/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Your max_length is set to 256, but you input_length is only 86. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 256, but you input_length is only 92. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 256, but you input_length is only 193. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Your max_length is set to 256, but you input_length is only 254. You might consider decreasing max_length manually, e.g. summarizer

rouge1 average : 0.1223548369492104
rouge2 average : 0.03007936547579612
rougeL average : 0.08099594851409213
rougeLs average : 0.08099594851409213
chrf average : 20.607812911329166


Technology Only

In [91]:
df_tech = pd.read_csv('Datasets/xl_sum_sample_test_technology.csv')

In [104]:
bart_untrained(df_tech)

Your max_length is set to 256, but you input_length is only 187. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=93)
Your max_length is set to 256, but you input_length is only 175. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=87)
Your max_length is set to 256, but you input_length is only 217. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=108)
Your max_length is set to 256, but you input_length is only 244. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=122)
Your max_length is set to 256, but you input_length is only 234. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=117)


rouge1 average : 0.26467407462886744
rouge2 average : 0.07949470580163175
rougeL average : 0.2153170488745322
rougeLs average : 0.2153170488745322
chrf average : 27.50620894169297


**Trained**

**All Categories: Model 0**

In [ ]:
!python3 transformers/examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path=facebook/bart-base \
    --do_train \
    --do_eval \
    --train_file 'w266_project/xl_sum_sample_train.csv' \
    --validation_file 'w266_project/xl_sum_sample_val.csv' \
    --text_column text \
    --summary_column summary \
    --push_to_hub=True \
    --max_source_length 128 \
    --max_target_length 32 \
    --num_train_epochs 1 \
    --per_device_train_batch_size=32 \
    --per_device_eval_batch_size=32 \
    --output_dir='w266_project/finetuned-BART-all-categories/model 0/finetuned-BART-all-categories' \
    --overwrite_output_dir=True \
    --predict_with_generate 

In [93]:
#remember to push new model to huggingface repo after running prev cell so that this cell uses the most recent model!!!!!!!

summarizer = pipeline("summarization", model="arisanguyen/finetuned-BART-all-categories", revision = 'model_0')

In [94]:
bart_r1 = []
bart_r2 = []
bart_rL = []
bart_rLs = []
bart_chrf = []

for i in range(int(len(df['text']))):
    
    #art = ' '.join(df['article'][i].split(' ')[:1024]) #truncated to first 1024 words, because that is all the model can handle
    
    candidate = summarizer(df['text'][i], 
                           truncation = True, #truncated to first 1024 words, because that is all the model can handle
                             #max_length=130, min_length=30, do_sample=False
                            )[0]
    candidate = [candidate['summary_text']]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    bart_r1.append(results['rouge1'])
    bart_r2.append(results['rouge2'])
    bart_rL.append(results['rougeL'])
    bart_rLs.append(results['rougeLsum'])
    
    results = chrf.compute(predictions=candidate,
                            references=ref)
    
    bart_chrf.append(results['score'])
    
#     if i in np.arange(0, 2000, 100):
#         data = {'rouge1': bart_r1, 'rouge2': bart_r2, 'rogueL': bart_rL, 'rogueLs': bart_rLs, 'chrf': bart_chrf}
#         scores = pd.DataFrame(data)
#         scores.to_csv(r'BART_trained_0_scores.csv', index=False)
#         print(i)
        
# data = {'rouge1': bart_r1, 'rouge2': bart_r2, 'rogueL': bart_rL, 'rogueLs': bart_rLs,'chrf': bart_chrf}
# scores = pd.DataFrame(data)
# scores.to_csv(r'BART_trained_0_scores.csv', index=False)

/home/ubuntu/w266/lib/python3.8/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Your max_length is set to 128, but you input_length is only 123. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)
Your max_length is set to 128, but you input_length is only 56. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 128, but you input_length is only 80. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)


In [95]:
print('Last Article', df['text'][i])
print('Last Reference Summary', ref)
print('Last Candidate Summary', candidate)

Last Article She tweeted that the clothes would be "archived & expertly cared for in the spirit & love of Michael Jackson, his bravery, & fans worldwide". The auction included a jacket worn during Jackson's Bad tour, that went for $240,000 (£148,000) and two crystal gloves. The items were all made by designers Dennis Tompkins and Michael Bush. Lady Gaga also tweeted a picture of herself and her bidding paddle at the auction. More than $5m (£3.1m) was raised by the sale, according to LA-based Julien's Auctions. Other items that went under the hammer included jackets from Michael Jackson's Dangerous and Thriller tours and a pair of jeans that went for $50,000 (£31,000). Some of the money raised by the auction is being donated to a guide dogs charity and a hospice in Las Vegas. American costume designers Michael Bush and Dennis Tompkins created thousands of original pieces for Michael Jackson during his career. However, despite Lady Gaga's assurances, some fans expressed their anger onlin

In [96]:
print('rouge1 average :', np.mean(bart_r1))
print('rouge2 average :', np.mean(bart_r2))
print('rougeL average :', np.mean(bart_rL))
print('rougeLs average :', np.mean(bart_rLs))
print('chrf average :', np.mean(bart_chrf))

rouge1 average : 0.3192639042917483
rouge2 average : 0.10466960084268205
rougeL average : 0.24501046685959502
rougeLs average : 0.24501046685959502
chrf average : 27.774860738085867


**All Categories: Model 1**

Added revision version. Set the max source length to 512, target length to 256, and number of epochs to 62 (~2k steps total) to be same as PEGASUS. 

In [ ]:
!python3 transformers/examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path=facebook/bart-base \
    --do_train \
    --train_file 'w266_project/xl_sum_sample_train.csv' \
    --text_column text \
    --summary_column summary \
    --max_source_length 512 \
    --max_target_length 256 \
    --num_train_epochs 62 \
    --per_device_train_batch_size=32 \
    --push_to_hub=True \
    --output_dir='w266_project/finetuned-BART-all-categories/model 1/finetuned-BART-all-categories' \
    --overwrite_output_dir=True \
    --predict_with_generate 

In [97]:
#remember to push new model to huggingface repo after running prev cell so that this cell uses the most recent model!!!!!!!

# model = AutoModel.from_pretrained(
#     "julien-c/EsperBERTo-small", revision="v2.0.1"  # tag name, or branch name, or commit hash
# )

summarizer = pipeline("summarization", model="arisanguyen/finetuned-BART-all-categories", revision = "model_1")

In [ ]:
get_default_args(summarizer)

In [101]:
%pwd
%cd ..

/home/ubuntu


In [103]:
df = pd.read_csv('./w266_project/Datasets/xl_sum_sample_val.csv')

bart_r1 = []
bart_r2 = []
bart_rL = []
bart_rLs = []
bart_chrf = []

for i in range(int(len(df['text']))):
    
    #art = ' '.join(df['article'][i].split(' ')[:1024]) #truncated to first 1024 words, because that is all the model can handle
    
    candidate = summarizer(df['text'][i], 
                           truncation = True, #truncated to first 1024 words, because that is all the model can handle
                           max_length = 256, # same as pegasus
                           min_length = 0, 
                             #max_length=130, min_length=30, do_sample=False
                            )[0]
    candidate = [candidate['summary_text']]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    bart_r1.append(results['rouge1'])
    bart_r2.append(results['rouge2'])
    bart_rL.append(results['rougeL'])
    bart_rLs.append(results['rougeLsum'])
    
    results = chrf.compute(predictions=candidate,
                            references=ref)
    
    bart_chrf.append(results['score'])
    
    if i in np.arange(0, 2000, 100):
        data = {'rouge1': bart_r1, 'rouge2': bart_r2, 'rogueL': bart_rL, 'rogueLs': bart_rLs, 'chrf': bart_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BART_trained_1_scores.csv', index=False)
        print(i)
        
data = {'rouge1': bart_r1, 'rouge2': bart_r2, 'rogueL': bart_rL, 'rogueLs': bart_rLs,'chrf': bart_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BART_trained_1_scores.csv', index=False)

print('Last Article', df['text'][i])
print('Last Reference Summary', ref)
print('Last Candidate Summary', candidate)

print('rouge1 average :', np.mean(bart_r1))
print('rouge2 average :', np.mean(bart_r2))
print('rougeL average :', np.mean(bart_rL))
print('rougeLs average :', np.mean(bart_rLs))
print('chrf average :', np.mean(bart_chrf))

/home/ubuntu/w266/lib/python3.8/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


0


Your max_length is set to 256, but you input_length is only 172. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=86)
Your max_length is set to 256, but you input_length is only 123. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)
Your max_length is set to 256, but you input_length is only 221. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=110)
Your max_length is set to 256, but you input_length is only 255. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=127)
Your max_length is set to 256, but you input_length is only 56. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 256, but you input_length is only 151. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=75)
Your max_length is set to 256, but you input_length is only 220. You might 

Last Article She tweeted that the clothes would be "archived & expertly cared for in the spirit & love of Michael Jackson, his bravery, & fans worldwide". The auction included a jacket worn during Jackson's Bad tour, that went for $240,000 (£148,000) and two crystal gloves. The items were all made by designers Dennis Tompkins and Michael Bush. Lady Gaga also tweeted a picture of herself and her bidding paddle at the auction. More than $5m (£3.1m) was raised by the sale, according to LA-based Julien's Auctions. Other items that went under the hammer included jackets from Michael Jackson's Dangerous and Thriller tours and a pair of jeans that went for $50,000 (£31,000). Some of the money raised by the auction is being donated to a guide dogs charity and a hospice in Las Vegas. American costume designers Michael Bush and Dennis Tompkins created thousands of original pieces for Michael Jackson during his career. However, despite Lady Gaga's assurances, some fans expressed their anger onlin

**All Categories: Model 2 (added beams, chagned epochs, added validation target length)**

In [29]:
%pwd
%cd ..
%pwd

/home/ubuntu


'/home/ubuntu'

In [37]:
!python3 transformers/examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path=facebook/bart-base \
    --do_train \
    --train_file 'w266_project/Datasets/xl_sum_sample_train.csv' \
    --text_column text \
    --summary_column summary \
    --max_source_length 512 \
    --max_target_length 256 \
    --val_max_target_length 256 \
    --num_beams 5 \
    --num_train_epochs 5 \
    --per_device_train_batch_size=32 \
    --push_to_hub=True \
    --output_dir='w266_project/finetuned-BART-all-categories/model 2/finetuned-BART-all-categories' \
    --overwrite_output_dir=True \
    --predict_with_generate 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/ubuntu/w266/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
04/07/2023 15:57:38 - WARNING - __main__ - Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
04/07/2023 15:57:38 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=No

In [40]:
#remember to push new model to huggingface repo after running prev cell so that this cell uses the most recent model!!!!!!!

summarizer = pipeline("summarization", model="arisanguyen/finetuned-BART-all-categories", revision = "model_2")

In [14]:
# hyperparameters defaults to huggingface defaults 
# https://huggingface.co/docs/transformers/main_classes/text_generation
def bart_scores(mod, data, do_sample = False, num_beams = 1, top_k = 50, num_beam_groups = 1):

    bart_r1 = []
    bart_r2 = []
    bart_rL = []
    bart_rLs = []
    bart_chrf = []
    texts = []
    references = []
    candidates = []

    for i in range(int(len(data['text']))):

        #art = ' '.join(df['article'][i].split(' ')[:1024]) #truncated to first 1024 words, because that is all the model can handle

        candidate = mod(data['text'][i], 
                               truncation = True, #truncated to first 1024 words, because that is all the model can handle
                               max_length = 256, # same as pegasus
                               min_length = 0, 
                               do_sample = do_sample,
                               num_beams = num_beams, 
                               top_k = top_k,
                               num_beam_groups = num_beam_groups,
                                )[0]
        candidate = [candidate['summary_text']]
        #pprint(candidate[0], compact=True)

        ref = [data['summary'][i]]

        references.append(ref)
        
        candidates.append(candidate)
        
        texts.append(data['text'][i])
        
        results = rouge.compute(predictions=candidate,
                                references=ref)

        bart_r1.append(results['rouge1'])
        bart_r2.append(results['rouge2'])
        bart_rL.append(results['rougeL'])
        bart_rLs.append(results['rougeLsum'])

        results = chrf.compute(predictions=candidate,
                                references=ref)

        bart_chrf.append(results['score'])
    
    print('Last Article', data['text'][i])
    print('Last Reference Summary', ref)
    print('Last Candidate Summary', candidate)

    print('rouge1 average :', np.mean(bart_r1))
    print('rouge2 average :', np.mean(bart_r2))
    print('rougeL average :', np.mean(bart_rL))
    print('rougeLs average :', np.mean(bart_rLs))
    print('chrf average :', np.mean(bart_chrf))
    
    data = {'articles': texts, 'refs': references, 'candidate':candidates, 'rouge1': bart_r1, 'rouge2': bart_r2, 'rogueL': bart_rL, 'rogueLs': bart_rLs,'chrf': bart_chrf}
    scores = pd.DataFrame(data)
    scores.to_csv(r'BART_untrained_scores.csv', index=False)
    
#     if i in np.arange(0, 2000, 100):
#         data = {'rouge1': bart_r1, 'rouge2': bart_r2, 'rogueL': bart_rL, 'rogueLs': bart_rLs, 'chrf': bart_chrf}
#         scores = pd.DataFrame(data)
#         scores.to_csv(r'BART_trained_0_scores.csv', index=False)
#         print(i)
        
# data = {'rouge1': bart_r1, 'rouge2': bart_r2, 'rogueL': bart_rL, 'rogueLs': bart_rLs,'chrf': bart_chrf}
# scores = pd.DataFrame(data)
# scores.to_csv(r'BART_trained_0_scores.csv', index=False)

In [56]:
bart_scores(summarizer, df)

Your max_length is set to 256, but you input_length is only 172. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=86)
Your max_length is set to 256, but you input_length is only 123. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)
Your max_length is set to 256, but you input_length is only 221. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=110)
Your max_length is set to 256, but you input_length is only 255. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=127)
Your max_length is set to 256, but you input_length is only 56. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 256, but you input_length is only 151. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=75)
Your max_length is set to 256, but you input_length is only 220. You might 

Last Article She tweeted that the clothes would be "archived & expertly cared for in the spirit & love of Michael Jackson, his bravery, & fans worldwide". The auction included a jacket worn during Jackson's Bad tour, that went for $240,000 (£148,000) and two crystal gloves. The items were all made by designers Dennis Tompkins and Michael Bush. Lady Gaga also tweeted a picture of herself and her bidding paddle at the auction. More than $5m (£3.1m) was raised by the sale, according to LA-based Julien's Auctions. Other items that went under the hammer included jackets from Michael Jackson's Dangerous and Thriller tours and a pair of jeans that went for $50,000 (£31,000). Some of the money raised by the auction is being donated to a guide dogs charity and a hospice in Las Vegas. American costume designers Michael Bush and Dennis Tompkins created thousands of original pieces for Michael Jackson during his career. However, despite Lady Gaga's assurances, some fans expressed their anger onlin

**All Categories: Model 3 (using one of the best models above, changing the hyperparamters) THIS IS THE BEST OVERALL**

In [106]:
summarizer = pipeline("summarization", model="arisanguyen/finetuned-BART-all-categories", revision = "model_2")

In [108]:
bart_scores(summarizer, df, do_sample = True, num_beams = 4, top_k = 75)

Your max_length is set to 256, but you input_length is only 172. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=86)
Your max_length is set to 256, but you input_length is only 123. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)
Your max_length is set to 256, but you input_length is only 221. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=110)
Your max_length is set to 256, but you input_length is only 255. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=127)
Your max_length is set to 256, but you input_length is only 56. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 256, but you input_length is only 151. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=75)
Your max_length is set to 256, but you input_length is only 220. You might 

Last Article She tweeted that the clothes would be "archived & expertly cared for in the spirit & love of Michael Jackson, his bravery, & fans worldwide". The auction included a jacket worn during Jackson's Bad tour, that went for $240,000 (£148,000) and two crystal gloves. The items were all made by designers Dennis Tompkins and Michael Bush. Lady Gaga also tweeted a picture of herself and her bidding paddle at the auction. More than $5m (£3.1m) was raised by the sale, according to LA-based Julien's Auctions. Other items that went under the hammer included jackets from Michael Jackson's Dangerous and Thriller tours and a pair of jeans that went for $50,000 (£31,000). Some of the money raised by the auction is being donated to a guide dogs charity and a hospice in Las Vegas. American costume designers Michael Bush and Dennis Tompkins created thousands of original pieces for Michael Jackson during his career. However, despite Lady Gaga's assurances, some fans expressed their anger onlin

**Testing best model from experiments above**

In [6]:
summarizer = pipeline("summarization", model="arisanguyen/finetuned-BART-all-categories", revision = "model_2")

In [12]:
bart_scores(summarizer, dft, do_sample = True, num_beams = 4, top_k = 75)

Your max_length is set to 256, but you input_length is only 86. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 256, but you input_length is only 92. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 256, but you input_length is only 193. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Your max_length is set to 256, but you input_length is only 254. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=127)
Your max_length is set to 256, but you input_length is only 187. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=93)
Your max_length is set to 256, but you input_length is only 226. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=113)
Your max_length is set to 256, but you input_length is only 178. You might c

Last Article By Alice EvansBBC News 1. Value for money For the men's World Cup last summer, your pockets had to be bulging with Russian roubles if you wanted a chance to see football come home (or not) with your own eyes. The best seats in the house for the France v Croatia men's final cost an eye-watering 66,000 roubles (£824.44). But don't despair. If you don't have a spare 800 quid jangling around, it's still possible for you to go to a World Cup final. The most expensive seats in the women's final at the Stade de Lyon on 7 July are more than 10 times cheaper than in the men's tournament at £75.12 (84EUR). At the Women's World Cup in France this year, you can actually watch a game for as little as £8.05 (9EUR). C'est magnifique, non? This pricing imbalance manifests itself in merchandise, too. Perhaps to Nike's credit, the official shirts of the men's and women's World Cups both cost £89.95. However, while it is free to add the name of a Lioness to the women's shirt, it costs £13 to

**Testing Technology Only Using Best Model from Above**

In [113]:
%pwd

'/home/ubuntu'

In [114]:
df_test = pd.read_csv('w266_project/Datasets/xl_sum_sample_test_technology.csv')

In [115]:
summarizer = pipeline("summarization", model="arisanguyen/finetuned-BART-all-categories", revision = "model_2")

In [116]:
bart_scores(summarizer, df_test, do_sample = True, num_beams = 4, top_k = 75)

/home/ubuntu/w266/lib/python3.8/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Your max_length is set to 256, but you input_length is only 187. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=93)
Your max_length is set to 256, but you input_length is only 175. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=87)
Your max_length is set to 256, but you input_length is only 217. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=108)
Your max_length is set to 256, but you input_length is only 244. You might consider decreasing max_length manually, e.g. summari

Last Article She tweeted that the clothes would be "archived & expertly cared for in the spirit & love of Michael Jackson, his bravery, & fans worldwide". The auction included a jacket worn during Jackson's Bad tour, that went for $240,000 (£148,000) and two crystal gloves. The items were all made by designers Dennis Tompkins and Michael Bush. Lady Gaga also tweeted a picture of herself and her bidding paddle at the auction. More than $5m (£3.1m) was raised by the sale, according to LA-based Julien's Auctions. Other items that went under the hammer included jackets from Michael Jackson's Dangerous and Thriller tours and a pair of jeans that went for $50,000 (£31,000). Some of the money raised by the auction is being donated to a guide dogs charity and a hospice in Las Vegas. American costume designers Michael Bush and Dennis Tompkins created thousands of original pieces for Michael Jackson during his career. However, despite Lady Gaga's assurances, some fans expressed their anger onlin